In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation # animaciones
from IPython.display import HTML

from scipy.sparse import diags, kron, csr_matrix, csc_matrix, identity, eye
from scipy.sparse.linalg import spsolve, norm, inv, eigsh

import time